In [2]:
import numpy as np
import pandas as pd
import numpy.random as rd
from scipy import stats
import matplotlib as mpl
import itertools as itertools
import math
import matplotlib.mlab as mlab
from matplotlib.ticker import LogFormatterSciNotation
from matplotlib.ticker import ScalarFormatter
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import pyplot as plt
import matplotlib.animation as animation
#from matplotlib import font_manager
import matplotlib.mathtext as mathtext
from decimal import Decimal

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Aug 29 11:27:54 2018
@author: D. Dragomir
"""
#%matplotlib notebook
import glob
import numpy as np
#import tkinter
import matplotlib
#try:
#    matplotlib.use('TkAgg')
#except ImportError:
#    print("TkAgg error did occur")
from matplotlib import pyplot as plt
from matplotlib import gridspec
from scipy.signal import medfilt
import pandas as pd
# from tsearchutils import tsearchutils as tsu
#from mostutils import Processing
import astropy.io.fits as apf
import sys
import getpass
import emcee
import corner
from multiprocessing import Pool
#this is currenlty not working on my OS but might work for you
#import gaiarad as gr# this is the old way to get stellar radius, you can edit this out
# import updatevet_2min as uv#this is for making advanced DV reports, you can edit this out
import os
from pathlib import Path
import random as rd
import time as tm
import math

# from pyts.image import GASF, GADF, MTF, RecurrencePlots


In [9]:
all_known_TOIS = pd.read_csv('/Users/mharris/Downloads/csv-file-toi-catalog_03032022.csv', skiprows = 4)

In [18]:
all_known_TOIS = all_known_TOIS.drop_duplicates('TIC').reset_index(drop=True)
all_known_TOIS = all_known_TOIS.rename(columns={'TIC': 'TICID'})

In [13]:
TOIs_and_sectors = pd.read_csv('/Users/mharris/Downloads/all_TOIs_sectors_observed.csv')

In [23]:
all_known_TOIS = all_known_TOIS[['TICID', 'Full TOI ID']]

In [28]:
all_known_TOIS[all_known_TOIS['TICID'] == 2876]

,TICID,Full TOI ID
3146,2876,3131.01


In [29]:
TOIs_and_sectors

,TICID,Sectors
0,2876,11;38
1,4711,11;38
2,17361,11;38
3,160023,65;11;38
4,857186,42
...,...,...
5276,1989628303,67;27
5277,1990842033,1;67;27
5278,2010186093,16;56
5279,2016376984,16;17;56;24;57


In [31]:
new_TOIs_and_sectors = pd.merge(all_known_TOIS , TOIs_and_sectors, on="TICID", how="left").sort_values('TICID')

new_TOIs_and_sectors.to_csv('/Users/mharris/Downloads/all_TOIs_sectors_observed.csv', index=False)

In [ ]:

def lnprob_trap(pars,t,flux,unc,tc): #calculates the log liklihood of these parameters
    depth  = pars[0] #pars = parameters array
    t0 = pars[1]
    T = pars[2] #duration
    tau = pars[3] #ingress/egress
    if depth > 1. or depth < 0.:
        #print('bad depth')
        return -np.inf
    if t0 > max(t)+1. or t0 < min(t)-1. or np.abs(t0-tc) > 1.:
        #print('bad t0')
        #print(t0,max(t),min(t),tc)
        return -np.inf
    if T/24. > 3. or T < tau or  T < 0: #SHOULD REVISIT THIS CRITERION DWS
        #print('bad T')
        return -np.inf
    if tau/24. > 1. or tau < 0.: #SHOULD REVISIT THIS CRITERION DWS
        #print('bad tau')
        return -np.inf

    flux_theo = tsu().trapmodel(depth,t,t0,T/24.,tau/24.)

    return 0.0-0.5*np.sum(((flux_theo-flux)/unc)**2.) 

def lnprob_box(pars,t,flux,unc,tc): 
    depth  = pars[0]
    t0 = pars[1]
    tau = pars[2]
    if depth > 1. or depth < 0.:
        #print('bad depth')
        return -np.inf
    if t0 > max(t)+1. or t0 < min(t)-1. or np.abs(t0-tc) > 1.:
        #print('bad t0')
        #print(t0,max(t),min(t),tc)
        return -np.inf
    if tau < 0.:
        #print('bad tau')
        return -np.inf

    flux_theo = tsu().boxmodel2(depth,t,t0,tau/24.)

    return 0.0-0.5*np.sum(((flux_theo-flux)/unc)**2.)

def get_tic_info(ticid):
#    infile = '/Users/svillan/Dropbox (MIT)/tics/sb_ticradec_exo_CTL_07.02xTIC_v7.csv'
#    infile = '/Users/svillan/Dropbox (MIT)/tics/si_ticradec_exo_CTL_07.02xTIC_v7.csv'
    #if curmag == '10': magbin = 'b'
    #if curmag == '11': magbin = 'i'
    #if curmag == '12': magbin = 'f'
    #infile = '/Users/svillan/Dropbox (MIT)/tics/s'+magbin+'_ticradec_exo_CTL_07.02xTIC_v7.csv'
    #infile = '/Users/svillan/Dropbox (MIT)/tics/exo_CTL_08.01xTIC_v8.csv'

    infile = '/Volumes/MALH_grad_school/tic8.csv' 
    dframe = pd.read_csv(infile,header=1,usecols=[0,13,14,30,42,60,64,65,70,71,72,73],names=['tic','RA','DEC','Vmag','Jmag','Tmag','Teff','eTeff','Rad','eRad','Mass','eMass'],dtype={'tic':'str' , 'RA':'float' , 'DEC':'float' , 'Vmag':'float', 'Jmag':'float','Tmag':'float','Teff':'float','eTeff':'float','Rad':'float','eRad':'float','Mass':'float','eMass':'float'},index_col=False)
    match  = np.where(str(ticid) == dframe['tic'])
    #print(match)
    if len(match) != 1:
    #if match.size <= 0:
        print('error in tic match')
    #elif np.isfinite(match[0]) != True:
    #elif np.isfinite(match[0]) != True:
        #print('error in the tic match')
    else:
        ra    = float(dframe['RA'][match[0]])
        dec   = float(dframe['DEC'][match[0]])
        Vmag  = float(dframe['Vmag'][match[0]])
        Jmag  = float(dframe['Jmag'][match[0]])
        Tmag  = float(dframe['Tmag'][match[0]])
        Teff  = float(dframe['Teff'][match[0]])
        eTeff = float(dframe['eTeff'][match[0]])
        Rad   = float(dframe['Rad'][match[0]])
        eRad  = float(dframe['eRad'][match[0]])
        Mass  = float(dframe['Mass'][match[0]])
        eMass = float(dframe['eMass'][match[0]])

    return ra,dec,Vmag,Jmag,Tmag,Teff,eTeff,Rad,eRad,Mass,eMass

def get_gaia_rad(ticid,ra,dec,path): #Don't need this function
    #search gaia and create the output in a csv file 
    #get best stellar radius
    rad,rad_lo,rad_hi = gr.gaiarad(ticid,ra,dec,path)
    return rad,rad_lo,rad_hi

def period_search(pars,t,flux,unc): #DWS what does this function do?--Email
    depth  = pars[0]
    t0 = pars[1]
    T = pars[2]
    #tau = pars[3]

    periodgrid = 1./np.linspace(1./35., 1./0.5, 35000) 
    lnprobper = np.zeros(len(periodgrid))
    print('running period search')
    for i in range(len(periodgrid)):
        phase = tsu().phasefold(t,periodgrid[i],t0) #This function is in tsearchutils
        flux_theo = tsu().boxmodel2(depth,phase,0.5,(T/24.)/periodgrid[i]) 
        lnprobper[i] = 0.0-0.5*np.sum(((flux_theo-flux)/unc)**2.)

    print('done with search')
    bestndx = np.where(lnprobper == max(lnprobper))
    bestper0 = periodgrid[bestndx]
    bestper = bestper0[0]
    maxtime = np.max(t)-np.min(t)
    good = np.array(np.where(periodgrid > maxtime))[0]
    if good.size == 0:
        print('period results bad')
        periodic = False
        bestper = maxtime
        probmin = np.max(lnprobper)
    else:
        probmax = np.max(lnprobper[good])
        probmin = np.min(lnprobper[good])
        delprob = np.abs(probmax-probmin)

        if max(lnprobper) >= probmax+delprob and bestper <= maxtime:
            periodic = True
        else:
            periodic = False

    #plt.close()
    #plt.scatter(periodgrid,lnprobper)
    #plt.axhline(probmin)
    #plt.axhline(np.max(lnprobper))
    #plt.axvline(bestper)
    #plt.axvline(maxtime)
    #plt.show()
    #plt.close()



    phase = tsu().phasefold(t,bestper,t0)
    phase_theo = np.linspace(0.,1.,1000)
    #flux_theo = tsu().trapmodel(depth,phase_theo,0.5,(T/24.)/bestper,(tau/24.)/bestper)
    flux_theo = tsu().boxmodel2(depth,phase_theo,0.5,(T/24.)/bestper)
    model_full_time = np.linspace(np.min(t),np.max(t),1000)
    model_full_phase = tsu().phasefold(model_full_time,bestper,t0)
    #model_full_flux = tsu().trapmodel(depth,model_full_phase,0.5,(T/24.)/bestper,(tau/24.)/bestper)
    model_full_flux = tsu().boxmodel2(depth,model_full_phase,0.5,(T/24.)/bestper)
    model_single_time = np.linspace(0.,1.,1000)
    model_single_flux = tsu().boxmodel2(depth,model_single_time,0.5,(T/24.)/bestper)
    return bestper,phase,model_full_time,model_full_flux,model_single_time,model_single_flux,periodic,periodgrid,lnprobper

def remove_elements_based_on_indxs_fn(lst_of_lsts, remove_indxs):
    #returns arrays, also can take arrays in lst_of_lsts
    new_lst_of_lst = []
    for lst in lst_of_lsts:
        lst = [i for j, i in enumerate(lst) if j not in remove_indxs]
        new_lst_of_lst.append(np.array(lst))
    return new_lst_of_lst


def binning_function_based_on_median(data, pts_per_bin):
    binned_data = []
    for i in range(0, len(data), pts_per_bin):
        if i+pts_per_bin<=len(data):
           # print(data[i:i+pts_per_bin])
            binned_data.append(np.median(data[i:i+pts_per_bin]))
        else:
            binned_data.append(np.median(data[i:]))
    return binned_data

def binning_several_lists_at_once(data_list, pts_per_bin):
    return [binning_function_based_on_median(x, pts_per_bin) for x in data_list]


def new_binning_function_by_time(time, flux, raw, time_size_of_bins):
    time = np.array(time).byteswap().newbyteorder() 
    flux = np.array(flux).byteswap().newbyteorder() 
    raw = np.array(raw).byteswap().newbyteorder() 

    interval = time_size_of_bins/60./24.
    df = pd.DataFrame({'time': time, 'flux':flux, 'raw':raw})
    df = df.append(df)
    #print(df)
    numbins = np.array(list(range(int(np.ceil((max(time)-min(time))/interval))+1)))
    #print(df)
    bins = np.array([min(time)+x*interval for x in numbins])
    #print(bins)
    df['time_bins'] = pd.cut(df.time, bins)
    new_time = [x for x in df.groupby('time_bins').mean()['time'] if not math.isnan(x)]
    new_flux = [x for x in df.groupby('time_bins').mean()['flux'] if not math.isnan(x)]
    new_raw  = [x for x in df.groupby('time_bins').mean()['raw'] if not math.isnan(x)]
    #print(len(new_time), len(new_flux), len(new_raw))
    return [np.array(new_time), np.array(new_flux), np.array(new_raw)]



In [11]:
usr = str(getpass.getuser())

#np.set_printoptions(threshold=np.nan)
#pd.set_option('display.max_rows', 10000)


matplotlib.rcParams['font.family'] = "serif"
matplotlib.rcParams['xtick.minor.visible'] = "True"
matplotlib.rcParams['ytick.minor.visible'] = "True"
matplotlib.rcParams['xtick.direction'] = "in"
matplotlib.rcParams['ytick.direction'] = "in"
matplotlib.rcParams['xtick.top'] = 'True'
matplotlib.rcParams['ytick.right'] = 'True'
matplotlib.rcParams['axes.axisbelow'] = 'True'


#~sector starts, Julian date - TESS launch date (2457000)
#01 1324
#02 1353.5
#03 1383
#04 1409
#05 1437 #1436.9 from DD
#06 1466
#07 1491
#08 1517
#09 1543.5
#10 1569.5
#11 1590
#12 1625
#13 1653
#14 1683
#15 1711
#16 1738.4
#17 1764.6
#18 1790.6
#19 1816.0

secstart = 1436.9
secend = secstart+28.
cursec = '5'
#curmag = '10'

#Directories
#indir = '/Users/'+usr+'/Dropbox (MIT)/TESS/qlp_lcs/'
#outdir = '/Users/svilmasteroutdir = '/home/bobby/mallory/Pipeline/Output_files_2min/'

    
indir = masterindir+'sector_'+cursec+'/' #'/pdo/ramp/tev/qlp-delivery/sector-1/sector-1_first_delivery/'




# #for Missed singles
# missed_singles_masteroutdir = '/home/bobby/mallory/Pipeline/Output_files/SAMPLE_out/'
# if os.path.exists('/home/bobby/mallory/Pipeline/Output_files/SAMPLE_out/sector_'+cursec)==False:
#     sec_outdir = os.path.join(missed_singles_masteroutdir, 'sector_'+cursec)
#     os.mkdir(sec_outdir)
# outdir = '/home/bobby/mallory/Pipeline/Output_files/SAMPLE_out/sector_'+cursec+'/'


#read in TESS light curves 
#QLP TCE
infiles = []
for root, dirs, files in os.walk('.'):
    for file in files:
        if(file.endswith(".fits")):
            infiles.append(os.path.join(root,file))
infiles = sorted(infiles)
tics = len(infiles)
#print("infiles", infiles)

#SPOC all
#infiles = sorted(glob.glob(indir+'light-curve/*fits.gz'))
detectsigspec = 5.0 #5.0 #3.0 #with the current setup, a threshold too high could kill long-duration events. #BLS spectrum #something to change later
seclength = 27.4
#knownsingles = [290165539,139147770,394287035,52368076,214517366,139198430,150098860,152283270,53812109,270341214,309402106,444151146,141428309,177350401,233964642]
#172,278,405,584,749

#get the pdumps ready
#pdir = '/Users/'+usr+'/Dropbox (MIT)/TESS/ts_lcs_pdump/' #Email Steven to get this file
pdir = '/../Mdwarf_trends_project/Pipeline/pdumps'

pfiles = sorted(glob.glob(pdir+'pdump_sector*.csv'))
#print(pfiles)                                                                                           
pdumps = []
for ppp in range(len(pfiles)):
    print("ppp", ppp)
    print(pfiles[ppp])
    df = pd.read_csv(pfiles[ppp])
    pdumps = np.append(pdumps,df['Time'])

    
#ctl = uv.readtic()
#get the csv file ready
outTIC   = np.array([])
outRA    = np.array([])
outDEC   = np.array([])
outTmag  = np.array([])
outVmag  = np.array([])
outJmag  = np.array([])
outTeff  = np.array([])
outRad   = np.array([])
outMass  = np.array([])
outTc    = np.array([])
outDepth = np.array([])
outDur   = np.array([])
outTau   = np.array([])
outRp    = np.array([])


NameError: name 'getpass' is not defined

In [ ]:
possible_TOIs = [
408232559,
415339577, 436640602]
#print(infiles)
tics = len(infiles) #len(candprops.star_tic) #runs through all the files
#    print('Nfiles = '+str(loopies))
new_infiles = []
new_tic = []

rd.seed(42)
#for i in np.random.choice(np.array(infiles), 1): 
for i in infiles:
    fileticid = np.int64(i.split("-")[2].lstrip("0")) #[4].lstrip("0"))
    if fileticid in possible_TOIs:
        new_infiles.append(i)
        new_tic.append(fileticid)

print(len(new_infiles))
print(new_tic)


In [ ]:
filename = new_infiles[0]
#print('filename', type(filename),filename)
fileticid = np.int64(filename.split("-")[2].lstrip("0")) #[4].lstrip("0"))
#         if zzz == 0: 
#             dict_of_ticids[fileticid] = []
#print(type(fileticid))
print('working on '+str(fileticid))

hdulist=apf.open(filename) #fits time series
tbdata = hdulist[1].data
#print(hdulist[1].header)
time = tbdata.field('TIME')
#print('got time')
raw  = tbdata.field('SAP_FLUX')
flux = tbdata.field('PDCSAP_FLUX') #tbdata.field('PDCSAP_FLUX') #tbdata.field('SAP_FLUX') #DSW - Email as if he's using lines 100-120
quality = tbdata.field('QUALITY') 
indxnan = np.logical_not((np.isnan(flux)) | (np.isnan(time))) #looking for where time or flux is nan
time = time[(indxnan) & (quality == 0)] #quality: flag 0=good, 1=bad
flux = flux[(indxnan) & (quality == 0)]
flux = flux/np.median(flux)
raw  = raw[(indxnan) & (quality == 0)]
#print('got all good quality data')

dur = np.max(time)-np.min(time)
     

cleant = time #[indx]
#        print("cleant1: ", cleant)
cleanf = flux #[indx]
cleanr = raw
#print(np.min(cleant),np.max(cleant),secstart,secend)
if len(cleant > 1):
    if (np.min(cleant) < secstart): #DWS
#                print("min cleant:", np.min(cleant), " secstart: ", secstart)
        cleanf = cleanf[cleant > secstart]
        cleanr = cleanr[cleant > secstart]
        cleant = cleant[cleant > secstart]
#                print("cleant2:", cleant)
if len(cleant > 1):
        if (np.max(cleant) > secend):
            cleanf = cleanf[cleant < secend]
            cleanr = cleanr[cleant < secend]
            cleant = cleant[cleant < secend]
#                    print("cleant3: ", cleant)

if len(cleant) <= 1:
    print('no good flux for '+str(fileticid))

newtime,newflux,newraw = cleant, cleanf, cleanr #RENAME later or rename mask newtime
print('median: ', np.median(newflux))
#        print("newtime: ",newtime)
#print('reassigned names')
binned_time, binned_flux, binned_raw = new_binning_function_by_time(newtime, newflux, newraw, 10)
#print(type(binned_time), type(newtime))
#spec,depth,specsh,depthsh = tsu().fitleastsq(binned_time,np.array(binned_flux)) #use this if still need to worry about outliers masquerading as BLS signals
spec,depth,specsh,depthsh = tsu().new_fitleastsq(binned_time,np.array(binned_flux)) #use this if still need to worry about outliers masquerading as BLS signals

#print('spec: ', len(spec), ' specsh: ', len(specsh))
#spec,depth = tsu().fitleastsq(newtime,np.array(newflux))
#time = np.array(cleants.BJD)  
detecthresh = detectsigspec*np.std(spec) 
print('standard dev: ', np.std(spec))
detecthresh_sh = detectsigspec*np.std(specsh) #use this if still need to worry about outliers masquerading as BLS signals
#print("spec for maxspec: ", spec)
maxspec = abs(np.max(spec)) #[(newtime > (np.min(newtime)+1.)) & (newtime < (np.max(newtime)-1.))])
indxmaxspec = np.where(spec > detecthresh)

a=0
#print("maxspec >= detecthresh: "+str(maxspec)+" >= "+str(detecthresh)+str(maxspec>detecthresh))
print('maxspec: ', maxspec, ', minspec: ', np.min(spec), ' detechthresh: ', detecthresh, ' detecthresh_sh', detecthresh_sh)
if (maxspec >= detecthresh): # & (np.any(rp < 30.)): #if SNR metric is larger than threshold AND rp is reasonable for a planetary object (but condition works only if we trust Rstar!)
#            a=1
#"""here is were the issue is occuring. why is maxspec always less than detectthresh?"""
#duration cut (as a way to better deal with single-point outliers without actually cropping a transit); but this could eliminate high-b and grazing transits
    goodtimes = np.array([],dtype = np.int16) #rename later for index
    for l in range(len(specsh)):
        #if zzz == 1:
            #print("need all True!: "+str(specsh[l] >= detecthresh_sh)+", "+str(l > 1)+", "+str(l < (len(specsh)-2)))
            #print(str(specsh[l])+" >= "+str(detecthresh_sh)+", and "+str(l)+" > 1, and "+str(l)+" < "+str(len(specsh)-2))
        if (specsh[l] >= detecthresh_sh) & (l > 1) & (l+3 < (len(specsh)-2)): #REVISIT: because the first and last points are likely to be systematic, we ignore first and last 2 points so they can't be considered as a transit
            #print('got to searching for a transit')
            if ([specsh[x]>= detecthresh_sh for x in range(l-1, l+2)]):#(specsh[l-1] >= detecthresh_sh) and (specsh[l+1] >= detecthresh_sh): #Revisit: looking for 3 points in a row

#                    if (specsh[l-1] >= detecthresh_sh) or (specsh[l+1] >= detecthresh_sh):
                a=1
                #dict_of_ticids[fileticid].extend([l-1,l,l+1])
               # dict_of_ticids[fileticid] = list(set(dict_of_ticids[fileticid]))
                goodtimes = np.append(goodtimes,int(l))
                print("This looks like a transit!")


In [ ]:
newunc = np.full(len(newflux),np.std(newflux)) #an array the size of newflux of the st.dev of newflux
binned_unc = np.full(len(binned_flux), np.std(binned_flux))
nwalkers = 40 
nchain = 1000
bchain = 500 #=0.5*nchain

indxbest = min(goodtimes[np.where(spec[goodtimes] == max(spec[goodtimes]))]) #DWS -email




tc_prior = float(binned_time[indxbest])
ini_box = np.array([1.-float(binned_flux[indxbest]),tc_prior,2.5],dtype=float) #10. for 30 minute data
ndim_box = len(ini_box)
err_box = np.array([0.1,0.1,.01])
pos_box = [ini_box+5e-3*err_box*np.random.randn(ndim_box) for rrr in range(nwalkers)]

#with Pool() as pool:
    #sampler = emcee.EnsembleSampler(nwalkers,ndim_box,lnprob_box,args=(newtime,newflux,newunc,tc_prior),pool=pool)
    #sampler.run_mcmc(pos_box,nchain,progress=True)

sampler = emcee.EnsembleSampler(nwalkers,ndim_box,lnprob_box,args=(binned_time,binned_flux,binned_unc,tc_prior))
sampler.run_mcmc(pos_box,nchain,progress=True)

samples_box = sampler.chain[:, bchain:, :].reshape((-1,ndim_box))
report_box = list(map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]), zip(*np.percentile(samples_box, [16, 50, 84],axis=0))))
pars_box_best = [report_box[0][0],report_box[1][0],report_box[2][0]]
box_best_depth = report_box[0][0]
box_best_depth_lo = report_box[0][1]
box_best_depth_hi = report_box[0][2]
box_best_t0 = report_box[1][0]
box_best_T = report_box[2][0]

#try:
#get tic ra,dec
#                        ra,dec = get_tic_info(str(fileticid),curmag)
    #RA,DEC,Vmag,Jmag,Tmag,Teff,eTeff,Rad,eRad,Mass,eMass = get_tic_info(str(fileticid))
#get gaia radius
#                        g_rad,g_rad_lo,g_rad_hi = get_gaia_rad(str(fileticid),ra,dec,outdir)
#                        g_rad_sig_lo = g_rad-g_rad_lo
#                        g_rad_sig_hi = g_rad_hi-g_rad
#except:
#                       g_rad=np.nan
    #Rad = np.nan
    #RA  = np.nan
    #DEC = np.nan
    #Vmag = np.nan
    #Tmag = np.nan
    #Jmag = np.nan
    #Teff = np.nan
    #Mass = np.nan


#rp/rs = sqrt(depth)
#rp = rs*sqrt(depth)

#p_rad = 109.*Rad*np.sqrt(trap_best_depth)



#check for periodicity
#bestper, phase, per_full_time, per_full_flux, phased_time, phased_flux, periodic, periodgrid, lnprobper = period_search(pars_trap_best,newtime,newflux,newunc)
bestper, phase, per_full_time, per_full_flux, phased_time, phased_flux, periodic, periodgrid, lnprobper = period_search(pars_box_best,binned_time,binned_flux,binned_unc)

#model_time = np.arange(np.min(newtime),np.max(newtime),5e-3)
model_time = per_full_time

if periodic == False:
    #model_trap_0 = tsu().trapmodel(pars_trap_best[0],model_time,pars_trap_best[1],pars_trap_best[2]/24.,pars_trap_best[3]/24.)
    #model_trap_best = (model_trap_0-1.)*1000.
    model_box_0 = tsu().boxmodel2(pars_box_best[0],model_time,pars_box_best[1],pars_box_best[2]/24.)
    model_box_best = (model_box_0-1.)*1000.
else:
    model_box_best = (per_full_flux-1.)*1000.

plt.gcf().clear() 
plt.suptitle(str(fileticid))
#plt.figure(1,constrained_layout=True)
plt.figure(1)
gs = gridspec.GridSpec(3,1,height_ratios = [1,1,3],hspace=0.5)

plt.subplot(gs[0])
x,y = binned_time, binned_raw
delx = max(x)-min(x)
dely = max(y)-min(y)
plt.axis([min(x)-0.01*delx, max(x)+0.01*delx, min(y)-0.01*dely, max(y)+0.01*dely])
plt.scatter(binned_time,binned_raw,s=0.1,color='C2')
for ppp in range(len(pdumps)):
    plt.axvline(x=pdumps[ppp],ymin=0,ymax=1,color='C3',alpha=0.1,zorder=-1)
plt.xlabel('TJD')
plt.ylabel('Raw Flux')
#plt.show()



plt.subplot(gs[1])
x,y = newtime, (newflux-1.)*1000.
delx = max(x)-min(x)
dely = max(y)-min(y)
plt.axis([min(x)-0.01*delx, max(x)+0.01*delx, min(y)-0.01*dely, max(y)+0.01*dely])

plt.plot(model_time,model_box_best,color='C1',alpha=0.8)
plt.scatter(newtime,(newflux-1.)*1000.,s=0.1,color='C0')
for ppp in range(len(pdumps)):
    plt.axvline(x=pdumps[ppp],ymin=0,ymax=1,color='C3',alpha=0.1,zorder=-1)
plt.xlabel('TJD')
plt.ylabel('Relative Flux [ppt]')
plt.subplot(gs[2])


In [ ]:
fig, ax = plt.subplots(figsize=(5,12))
transit_times = []
transit_flux = []
def animate_pipeline(l, transit_times):
    ax.clear()
    
    ax.scatter(newtime,(newflux-1.)*1000.,color='lightgrey',s=4,alpha=0.5,label='SPOC 2-min detrended',rasterized=True)
    ax.scatter(binned_time[l], (binned_flux[l]-1)*1000.,color='black',s=4,alpha=0.5,label='SPOC 2-min detrended, binned',rasterized=True)
    ax.axhline(-10, 10, detecthresh_sh, color='gray', linestyle='--', linewidth=3)

    if (specsh[l] >= detecthresh_sh) & (l > 1) & (l+3 < (len(specsh)-2)): #REVISIT: because the first and last points are likely to be systematic, we ignore first and last 2 points so they can't be considered as a transit
            #print('got to searching for a transit')
            if ([specsh[x]>= detecthresh_sh for x in range(l-2, l)]):#(specsh[l-1] >= detecthresh_sh) and (specsh[l+1] >= detecthresh_sh): #Revisit: looking for 3 points in a row
                transit_times.append(binned_time[l])
                transit_flux.append(binned_flux[l])
    ax.scatter(transit_times, transit_flux,color='limegreen',s=4,alpha=0.5,rasterized=True)
    for ppp in range(len(pdumps)):
        plt.axvline(x=pdumps[ppp],ymin=0,ymax=1,color='C1',zorder=-3)
    ax.legend(fontsize='small',loc=3)
    ax.tick_params(labelsize=8)
    ax.xlabel('Time [h]',fontsize='small')
    ax.ylabel('Normalized Flux',fontsize='small')
    ax.set_xlim([min(newtime), max(new_time)])

ani = animation.FuncAnimation(fig, animate_pipeline, list(range(len(binned_time))), interval=1000, repeat=False)
ani.save('pipeline_vid.mp4', fps=1, extra_args=['-vcodec', 'libx264'], savefig_kwargs={'facecolor':'black'})

plt.show()

